# Bigram-gram based LLM

In [1]:
import numpy as np
from datasets import load_dataset
import torch
import re

## Dataset Preprocessing

In [2]:
ds = load_dataset("MuskumPillerum/General-Knowledge")['train']

In [3]:
l = []
for i,j in zip(ds['Question'][:10000], ds['Answer'][:10000]):
    if i and j:
        # print(i)
        i = re.sub(r'[^a-zA-Z 0-9]', '',i.replace('\\n', "")) + ' ?'
        j = re.sub(r'[^a-zA-Z 0-9]', '',j.replace('\\n', ""))
        # j = j.replace('\\n', "")
        l.append(f'<Start> Question: {i.lower()} Answer: {j.lower()} <Stop>')

        # print(i)
        # print(j)


In [4]:
vocab = []
for sentence in l:
    vocab.extend(sentence.split())


In [5]:
vocab_dict = {} 
ct = 0
for i in set(vocab):
    vocab_dict[i] = ct
    ct += 1

In [6]:
len(vocab), len(vocab_dict)

(555927, 24466)

In [7]:
rev_vocab_dict = {v:k for k,v in vocab_dict.items()}

## Model Creation

In [8]:
n_gram_matrix = np.ones((len(vocab_dict), len(vocab_dict)), dtype=np.float32) # 15 gig!
n_gram_matrix.shape

(24466, 24466)

In [9]:
for sentence in l:
    prev_word_index = vocab_dict['<Start>']
    # print(sentence)
    for word in sentence.split()[1:]:
        index = vocab_dict[word]
        n_gram_matrix[prev_word_index][index] += 100 # So that it is bigger than 1
        prev_word_index = index


# Inference

In [11]:
out = vocab_dict['<Start>']
len = 0
while out != vocab_dict['<Stop>']:
    row = torch.tensor(n_gram_matrix[out], dtype=torch.float32)
    row = row/ row.sum()
    
    # print(row)
    next_index = torch.multinomial(row, num_samples=1, replacement=True)
    # print(next_index)
    print(rev_vocab_dict[next_index.item()], end=' ')
    out = next_index
    len += 1
    if len > 30: break

Question: what is a device drivers gurgle woodwork days pace snow ideally remedy enhance fuming aggregate asqrt45451245154518 rossini personalised options have led neem ? Answer: jeff openly 6561 screen boot pantry 